In [1]:
#连接ERP的DB2数据库，获取TACAIA2（前台：ACAI25）,TACACM4(ACAC80非产成品),TACACMM(ACACMM),TACACJ5(废钢入库履历)
import ibm_db
import pandas as pd
connStr='DATABASE=BHBXA1;HOSTNAME=192.168.2.28;PORT=50000;PROTOCOL=TCPIP;UID=bhbxa1;PWD=bhbxa1'
con=ibm_db.connect(connStr,'','')


In [2]:
import ibm_db_dbi

In [3]:
conn=ibm_db_dbi.Connection(con)

In [4]:
tacaia2=pd.read_sql('select * from tacaia2',conn,parse_dates={'APP_TRNC_TIME':'%Y%m%d%H%M%S'})

In [5]:
from pandas.api.types import CategoricalDtype
tacaia2['ACCOUNT_TITLE_ITEM']=tacaia2['ACCOUNT_TITLE_ITEM'].astype(CategoricalDtype(['01','33','03','31','50','51']))#这里对账务代码是01-33-03-31-50-51的顺序，因为先有产出这个材料，再有后续的专用充当和投入或销售，这里暂不考虑一个材料号被二次以上转用的情况，如上一个木材料转用经过工序加工后，转用产出一个材料号，当期这个材料又直接被专用挂了另外的订单号的情况
tacaia2['ACCOUNT_TITLE_ITEM'].cat.categories

Index(['01', '33', '03', '31', '50', '51'], dtype='object')

In [6]:
tacaia2.set_index(['ACCOUNT_TITLE_ITEM','MAT_NO','UNIT_CODE','SALE_ORDER_SUB_NO'],inplace=True) #将账务代码提到顺序第1位，以方便下面在同一投入账务代码下，按有投入的材料号迭代。

In [7]:
tacaia2.sort_index(level='MAT_NO',inplace=True) #如果按多层索引综合排序，不行，则单按账务代码排序

In [8]:
from numpy import nan as NA
tacaia2=tacaia2.dropna() #剔除第一行的空行

In [9]:
tacaia2['MAT_TYPE']=tacaia2['MAT_TYPE'].astype('int64')

In [10]:
tacaia2.insert(tacaia2.shape[1],'MAT_VALUE',0.00)
tacaia2.insert(tacaia2.shape[1],'MAT_PRICE',0.00)
tacaia2.insert(tacaia2.shape[1],'LOST_VALUE',0.00)
tacaia2.insert(tacaia2.shape[1],'LOST_PRICE',0.00)
tacaia2.insert(tacaia2.shape[1],'SALE_VALUE',0.00)
tacaia2.insert(tacaia2.shape[1],'SALE_PRICE',0.00)

In [54]:
def InputPrices(MAT_NO,PRODUCT_CODE,MTRL_NO,MAT_WT,ACC_TITLE):
    if PRODUCT_CODE[0] in ['0','1','2']:
        MatPrice=(tacacm4.loc[PRODUCT_CODE,'PRST_VRNC_AMOUNT']+tacacm4.loc[PRODUCT_CODE,'OTHR_INCM_VRNC_AMT']-tacacm4.loc[PRODUCT_CODE,'OTHER_SEND_VRNC_AMT']+tacacm4.loc[PRODUCT_CODE,'CRRT_OTPT_VRNC_AMT']+tacacm4.loc[PRODUCT_CODE,'IN_MATCH_VRNC_AMT'])/(tacacm4.loc[PRODUCT_CODE,'PERIOD_START_N']+tacacm4.loc[PRODUCT_CODE,'OTHERS_RECEIVE_N']-tacacm4.loc[PRODUCT_CODE,'OTHERS_SEND_N']+tacacm4.loc[PRODUCT_CODE,'CURRENT_N']+tacacm4.loc[PRODUCT_CODE,'IN_MATCH_N'])#投入材料单价=（期初金额+其他收入-其他支出+本期产出+本期转入）的金额/重量
    elif PRODUCT_CODE[0] in ['3','4','5','9']:
        MatPrice=(tacacmm.loc[PRODUCT_CODE,'START_AMOUNT']+tacacmm.loc[PRODUCT_CODE,'OTHR_INCM_ACT_AMT']-tacacmm.loc[PRODUCT_CODE,'OTHR_SEND_ACT_AMT']+tacacmm.loc[PRODUCT_CODE,'CRRT_OTPT_ACT_AMT']+tacacmm.loc[PRODUCT_CODE,'NEXT_MATCH_AMT']
)/(tacacmm.loc[PRODUCT_CODE,'PERIOD_START_N']+tacacmm.loc[PRODUCT_CODE,'OTHERS_RECEIVE_N']-tacacmm.loc[PRODUCT_CODE,'OTHERS_SEND_N']+tacacmm.loc[PRODUCT_CODE,'CURRENT_N']+tacacmm.loc[PRODUCT_CODE,'NEXT_MATCH_USED_N']
)#投入材料单价=（期初金额+其他收入-其他支出+本期产出+本期转入）的金额/重量
    else:
        raise Exception('MAT_TYPE not NORMAL!')
    MatValue=MatPrice*MAT_WT
    
    LostPercent=0.4#次板和二级品的损失百分比（以后如果要跟着系统参数表，到时可另从数据库取数，这里暂时设成常用的40%）
    #判断投入材料号为参数中材料号&物料形态数值大于3（即为二等品或废次）的计数是否大于0，是则分无聊形态为4和5、9计算损失金额；否则损失金额为0
    if tacaia2.loc['0' + ACC_TITLE[1]][(tacaia2.loc['0' + ACC_TITLE[1]].IN_MAT_NO==MAT_NO) &(tacaia2.loc['0' + ACC_TITLE[1]].MAT_TYPE>3)].count()['MAT_WT']>0:
        LostValue=tacaia2.loc['0' + ACC_TITLE[1]][(tacaia2.loc['0' + ACC_TITLE[1]].IN_MAT_NO==MAT_NO) & ((tacaia2.loc['0' + ACC_TITLE[1]].MAT_TYPE==4) | (tacaia2.loc['0' + ACC_TITLE[1]].MAT_TYPE==5))].sum()['MAT_WT']*MatPrice*LostPercent+tacaia2.loc['0' + ACC_TITLE[1]][(tacaia2.loc['0' + ACC_TITLE[1]].IN_MAT_NO==MAT_NO) &(tacaia2.loc['0' + ACC_TITLE[1]].MAT_TYPE==9)].sum()['MAT_WT']*(MatPrice-tacacj5.loc['900' + MTRL_NO + '000000','CRRT_YEAR_STD_AMT'])
    else:
        LostValue=0.00

    if tacaia2.loc['0' + ACC_TITLE[1]][(tacaia2.loc['0' + ACC_TITLE[1]].IN_MAT_NO==MAT_NO)&(tacaia2.loc['0' + ACC_TITLE[1]].MAT_TYPE<=3)].count()['MAT_WT']>0:
        LostPrice=LostValue/tacaia2.loc['0' + ACC_TITLE[1]][(tacaia2.loc['0' + ACC_TITLE[1]].IN_MAT_NO==MAT_NO)&(tacaia2.loc['0' + ACC_TITLE[1]].MAT_TYPE<=3)].sum()['MAT_WT']
        OtherLostValue=0#如果投入没有正品产出，需将材料损失汇总分摊到产出最多的产副品上。
    else:
        OtherLostValue=LostValue
        LostPrice=0.00
    return {
        'Mat_Value':MatValue,
        'Mat_Price':MatPrice,
        'Lost_Value':LostValue,
        'Lost_Price':LostPrice,
        'Other_Lost_Value':OtherLostValue
    }

In [28]:
def SalePrices(PRODUCT_CODE,MAT_WT):
    if PRODUCT_CODE[0] in ['0','1','2']:
        SalePrice=(tacacm4.loc[PRODUCT_CODE,'PRST_VRNC_AMOUNT']+tacacm4.loc[PRODUCT_CODE,'OTHR_INCM_VRNC_AMT']-tacacm4.loc[PRODUCT_CODE,'OTHER_SEND_VRNC_AMT']+tacacm4.loc[PRODUCT_CODE,'CRRT_OTPT_VRNC_AMT']+tacacm4.loc[PRODUCT_CODE,'IN_MATCH_VRNC_AMT']-tacacm4.loc[PRODUCT_CODE,'NEXT_PRSP_VRNC_AMT']-tacacm4.loc[PRODUCT_CODE,'NEXT_MATCH_VRNC_AMT'])/(tacacm4.loc[PRODUCT_CODE,'PERIOD_START_N']+tacacm4.loc[PRODUCT_CODE,'OTHERS_RECEIVE_N']-tacacm4.loc[PRODUCT_CODE,'OTHERS_SEND_N']+tacacm4.loc[PRODUCT_CODE,'CURRENT_N']+tacacm4.loc[PRODUCT_CODE,'IN_MATCH_N']-tacacm4.loc[PRODUCT_CODE,'NEXT_PRSP_USED_N']-tacacm4.loc[PRODUCT_CODE,'NEXT_MATCH_USED_N'])#销售材料单价=（期初金额+其他收入-其他支出+本期产出+本期转入-本期投入-本期转出）的金额/重量
    elif PRODUCT_CODE[0] in ['3','4','5','9']:
        SalePrice=(tacacmm.loc[PRODUCT_CODE,'START_AMOUNT']+tacacmm.loc[PRODUCT_CODE,'OTHR_INCM_ACT_AMT']-tacacmm.loc[PRODUCT_CODE,'OTHR_SEND_ACT_AMT']+tacacmm.loc[PRODUCT_CODE,'CRRT_OTPT_ACT_AMT']+tacacmm.loc[PRODUCT_CODE,'NEXT_MATCH_AMT']-tacacmm.loc[PRODUCT_CODE,'NEXT_PRSP_STD_AMT']-tacacmm.loc[PRODUCT_CODE,'NEXT_MATCH_AMT']
)/(tacacmm.loc[PRODUCT_CODE,'PERIOD_START_N']+tacacmm.loc[PRODUCT_CODE,'OTHERS_RECEIVE_N']-tacacmm.loc[PRODUCT_CODE,'OTHERS_SEND_N']+tacacmm.loc[PRODUCT_CODE,'CURRENT_N']+tacacmm.loc[PRODUCT_CODE,'NEXT_MATCH_USED_N']-tacacmm.loc[PRODUCT_CODE,'NEXT_PRSP_USED_N']-tacacmm.loc[PRODUCT_CODE,'NEXT_MATCH_USED_N']
)#销售材料单价=（期初金额+其他收入-其他支出+本期产出+本期转入-本期投入-本期转出）的金额/重量
    SaleValue=SalePrice*MAT_WT
    return {
        'Sale_Value':SaleValue,
        'Sale_Price':SalePrice
    }

In [29]:
def ProducePrices(INPUT_MAT,ACC_TITLE,MAT_TYPE,MAT_WT,MTRL_NO):
    #tacaia2.loc[('3'+ACC_TITLE[1],Input_MAT),'']
    if MAT_TYPE<=3:
        MatPrice=tacaia2.loc[('3'+ACC_TITLE[1],INPUT_MAT),'MAT_PRICE'][0]
        MatValue=MatPrice*MAT_WT
        LostPrice=tacaia2.loc[('3'+ACC_TITLE[1],INPUT_MAT),'LOST_PRICE'][0]
        LostValue=LostPrice*MAT_WT
    elif MAT_TYPE in [4,5]:
        LostPercent=0.4#次板和二级品的损失百分比（以后如果要跟着系统参数表，到时可另从数据库取数，这里暂时设成常用的40%）
        MatPrice=tacaia2.loc[('3'+ACC_TITLE[1],INPUT_MAT),'MAT_PRICE'][0]*(1-LostPercent)
        MatValue=MatPrice*MAT_WT
        LostPrice=0.00
        LostValue=LostPrice*MAT_WT
    elif MAT_TYPE==9:
        MatPrice=tacacj5.loc['900' + MTRL_NO + '000000','CRRT_YEAR_STD_AMT']
        MatValue=MatPrice*MAT_WT
        LostPrice=0.00
        LostValue=LostPrice*MAT_WT
    return {
        'Mat_Value':MatValue,
        'Mat_Price':MatPrice,        
        'Lost_Value':LostValue,
        'Lost_Price':LostPrice
    }

In [14]:
#tacaia2.sort_index(level=['ACCOUNT_TITLE_ITEM','MAT_NO','UNIT_CODE','SALE_ORDER_SUB_NO'],inplace=True) #跟着前面重设索引，把账务代码提到第1位，对应改变索引排序

In [15]:
#tacaia2.loc['31','KEY_SEQ']

In [16]:
#tacaia2.to_excel('tacaia2-202005.xlsx',sheet_name='tacaia2')
#tacaia2.loc[('B2003125AFA','LF03','33','MD2005061901'),'PRODUCT_CODE'][0][:4] #Pandas多层索引的引用
#[x[0] for x in tacaia2.index] #多层索引中一层所有元素的列表

In [17]:
acc_period=tacaia2['ACCOUNT_PERIOD'][1]
print(acc_period)

202007


In [18]:
tacacm4=pd.read_sql('select * from tacacm4 where YEAR=? and MON=?',conn,params=[acc_period[:4],acc_period[4:]])
tacacm4.set_index(['PRODUCT_CODE'],inplace=True)
tacacm4.drop(['REC_CREATOR', 'REC_CREATE_TIME', 'REC_REVISOR', 'REC_REVISE_TIME','ARCHIVE_FLAG', 'COMPANY_CODE', 'COMPANY_CNAME','COST_CENTER', 'STD_PRICE',"CURRNT_STD_AMOUNT","CRRT_OTPT_STD_AMT","OTHR_INCM_STD_AMT","SALE_STD_AMOUNT","NEXT_PRSP_STD_AMT","NEXT_MATCH_STD_AMT","OTHER_SEND_STD_AMT","PERIOD_END_STD_AMT","IN_MATCH_STD_AMT"],axis=1,inplace=True)
tacacm4[["CURRENT_N","CRRT_OTPT_VRNC_AMT","SALE_N","SALE_VRNC_AMOUNT","NEXT_PRSP_USED_N","NEXT_PRSP_VRNC_AMT","NEXT_MATCH_USED_N","NEXT_MATCH_VRNC_AMT","PERIOD_END_N","PERIOD_END_VRNC_AMT","IN_MATCH_N","IN_MATCH_VRNC_AMT"]]=[0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00]

In [19]:
#tacacmm.to_excel('tacacmm-202006.xlsx')

In [20]:
#tacacm4[['PERIOD_END_N','PERIOD_END_VRNC_AMT']]#test

In [21]:
#tacacm4[['PERIOD_END_N','PERIOD_END_VRNC_AMT']]=[0.00,0.00]
#tacacm4[['PERIOD_END_N','PERIOD_END_VRNC_AMT']]#test

In [22]:
tacacm4.columns#test

Index(['YEAR', 'MON', 'PERIOD_START_N', 'PRST_VRNC_AMOUNT', 'CURRENT_N',
       'CRRT_OTPT_VRNC_AMT', 'OTHERS_RECEIVE_N', 'OTHR_INCM_VRNC_AMT',
       'SALE_N', 'SALE_VRNC_AMOUNT', 'NEXT_PRSP_USED_N', 'NEXT_PRSP_VRNC_AMT',
       'NEXT_MATCH_USED_N', 'NEXT_MATCH_VRNC_AMT', 'OTHERS_SEND_N',
       'OTHER_SEND_VRNC_AMT', 'PERIOD_END_N', 'PERIOD_END_VRNC_AMT',
       'IN_MATCH_N', 'IN_MATCH_VRNC_AMT'],
      dtype='object')

In [23]:
tacacmm=pd.read_sql('select * from tacacmm where YEAR=? and MON=?',conn,params=[acc_period[:4],acc_period[4:]])
tacacmm.set_index(['PRODUCT_CODE'],inplace=True)
tacacmm.drop(['REC_CREATOR', 'REC_CREATE_TIME', 'REC_REVISOR', 'REC_REVISE_TIME','ARCHIVE_FLAG', 'COMPANY_CODE', 'COMPANY_CNAME', 'STD_PRICE'],axis=1,inplace=True)
tacacmm[['CURRENT_N','CRRT_OTPT_ACT_AMT','NEXT_MATCH_AMT', 'NEXT_MATCH_USED_N', 'GM_N', 'GM_ACT_AMT', 'SY_N','SY_ACT_AMT', 'LL_N','LL_ACT_AMT','PERIOD_END_N', 'PERIOD_END_ACT_AMT','NEXT_PRSP_USED_N','NEXT_PRSP_STD_AMT', 'IN_MATCH_AMT', 'IN_MATCH_N']]=[0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00]

In [24]:
tacacj5=pd.read_sql_query('select * from tacacj5 where YEAR_MON=?',conn,params=[acc_period])
tacacj5.set_index(['PRODUCT_CODE'],inplace=True)
tacacj5.drop(['REC_CREATOR', 'REC_CREATE_TIME', 'REC_REVISOR', 'REC_REVISE_TIME','ARCHIVE_FLAG', 'COMPANY_CODE', 'COMPANY_CNAME'],axis=1,inplace=True)

In [28]:
print(tacacj5.head())
print(tacacm4.head())
print(tacacmm.head())

YEAR_MON PRODUCT_CODE_CNAME  CRRT_YEAR_STD_AMT
PRODUCT_CODE                                               
9001000000     202007                                3.5398
9001100000     202007                                3.5398
9002000000     202007                                4.1592
9002400000     202007                                4.1592
9002405893     202007                                4.1592
              YEAR MON  PERIOD_START_N  PRST_VRNC_AMOUNT  CURRENT_N  \
PRODUCT_CODE                                                          
0003100002    2020  07         6989.00          89829.95        0.0   
0003100003    2020  07         5386.00          67926.77        0.0   
0003100004    2020  07        78769.37         985215.95        0.0   
0003100006    2020  07        48716.03         578597.51        0.0   
0003100719    2020  07         4138.88          54112.68        0.0   

              CRRT_OTPT_VRNC_AMT  OTHERS_RECEIVE_N  OTHR_INCM_VRNC_AMT  \
PRODUCT_CODE         

In [29]:
#将ACAI25表按1、材料号升序；2、账务代码：按33 03 31 01 50 51 的顺序排列；3、销售合同号按升序排列
#tacaia2[tacaia2['MAT_WT'].rank(method='dense',ascending=False)==3]

In [30]:
#对账务代码自定义排序：先设排序的列表；将账务代码列的数据类型改成‘category’（astype)；用set_category，设置账务代码的顺序；用sort_values使排序生效，同时，也将材料号和销售合同升序排列
'''acc_ttl_list=['33','03','31','01','50','51']
tacaia2['ACCOUNT_TITLE_ITEM']=tacaia2['ACCOUNT_TITLE_ITEM'].astype('category')
tacaia2['ACCOUNT_TITLE_ITEM'].cat.set_category(acc_ttl_list,inplace=True)
tacaia2.sortvalues(['MAT_NO','ACCOUNT_TITLE_ITEM','SALE_ORDER_SUB_NO'],ascending=True,inplace=True)
tacaia2'''

"acc_ttl_list=['33','03','31','01','50','51']\ntacaia2['ACCOUNT_TITLE_ITEM']=tacaia2['ACCOUNT_TITLE_ITEM'].astype('category')\ntacaia2['ACCOUNT_TITLE_ITEM'].cat.set_category(acc_ttl_list,inplace=True)\ntacaia2.sortvalues(['MAT_NO','ACCOUNT_TITLE_ITEM','SALE_ORDER_SUB_NO'],ascending=True,inplace=True)\ntacaia2"

In [1]:
def WriteSFC(PRODUCT_CODE,ACC_TITLE,MAT_VALUE,LOST_VALUE,SALE_VALUE,WT):
    if PRODUCT_CODE[0] in ['0','1','2']:
        SFCColumnName={
            'df':tacacm4,
            'Num':{
                '01':'CURRENT_N',
                '33':'NEXT_MATCH_USED_N',
                '03':'IN_MATCH_N',
                '31':'NEXT_PRSP_USED_N',
                '50':'SALE_N',
                '51':'SALE_N' 
            },
            'Value':{
                '01':'CRRT_OTPT_VRNC_AMT',
                '33':'NEXT_MATCH_VRNC_AMT',
                '03':'IN_MATCH_VRNC_AMT',
                '31':'NEXT_PRSP_VRNC_AMT',
                '50':'SALE_VRNC_AMOUNT',
                '51':'SALE_VRNC_AMOUNT'
            }
            
        }
        
    elif PRODUCT_CODE[0] in ['3','4','5','9']:
        SFCColumnName={
            'df':tacacmm,
            'Num':{
                '01':'CURRENT_N',
                '33':'NEXT_MATCH_USED_N',
                '03':'IN_MATCH_N',
                '31':'NEXT_PRSP_USED_N',
                '50':'GM_N',
                '51':'SY_N' 
            },
            'Value':{
                '01':'CRRT_OTPT_ACT_AMT',
                '33':'NEXT_MATCH_AMT',
                '03':'IN_MATCH_AMT',
                '31':'NEXT_PRSP_STD_AMT',
                '50':'GM_ACT_AMT',
                '51':'SY_ACT_AMT'
            }
        }

    SFCValue={
            '01':MAT_VALUE+LOST_VALUE,
            '03':MAT_VALUE+LOST_VALUE,
            '31':MAT_VALUE,
            '33':MAT_VALUE,
            '50':SALE_VALUE,
            '51':SALE_VALUE
        }
    SFCColumnName['df'].loc[PRODUCT_CODE,SFCColumnName['Num'][ACC_TITLE]]+=WT
    SFCColumnName['df'].loc[PRODUCT_CODE,SFCColumnName['Value'][ACC_TITLE]]+=SFCValue[ACC_TITLE]

In [50]:
#tacaia2.index.get_level_values('MAT_NO')[0]==' '  #因为排序后第一行[0]为空行，所以迭代时从第二行[1]开始（上面已删除空行，不用从第二行开始了）
OTHER_LOST_VALUE=0.00
n=0
for mat in tacaia2.index.get_level_values('MAT_NO'):
    for acc in ['01','33','03','31','50','51']:
        print('No.' + str(n))
        n=n+1
        try:
            print(tacaia2.loc[(acc,mat),'KEY_SEQ'])
            if acc=='01':
                tacaia2.loc[(acc,mat),['MAT_VALUE','MAT_PRICE','LOST_VALUE','LOST_PRICE']]=list(ProducePrices(tacaia2.loc[(acc,mat),'IN_MAT_NO'][0],acc,tacaia2.loc[(acc,mat),'MAT_TYPE'][0],tacaia2.loc[(acc,mat),'MAT_WT'][0],tacaia2.loc[(acc,mat),'MTRL_NO'][0]).values())
            if acc=='33':
                if tacaia2.loc[(acc,mat),'DIVVY_FLAG'][0]=='0':
                    INPUT_VALUE=list(InputPrices(mat,tacaia2.loc[(acc,mat),'PRODUCT_CODE'][0],tacaia2.loc[(acc,mat),'MTRL_NO'][0],tacaia2.loc[(acc,mat),'MAT_WT'][0],acc).values())
                    tacaia2.loc[(acc,mat),['MAT_VALUE','MAT_PRICE','LOST_VALUE','LOST_PRICE']]=INPUT_VALUE[:4]
                    OTHER_LOST_VALUE=OTHER_LOST_VALUE+INPUT_VALUE[4]
            if acc=='03':
                if tacaia2.loc[(acc,mat),'DIVVY_FLAG'][0]=='0':
                    tacaia2.loc[(acc,mat),['MAT_VALUE','MAT_PRICE','LOST_VALUE','LOST_PRICE']]=list(ProducePrices(tacaia2.loc[(acc,mat),'IN_MAT_NO'][0],acc,tacaia2.loc[(acc,mat),'MAT_TYPE'][0],tacaia2.loc[(acc,mat),'MAT_WT'][0],tacaia2.loc[(acc,mat),'MTRL_NO'][0]).values())
            if acc=='31':
                INPUT_VALUE=list(InputPrices(mat,tacaia2.loc[(acc,mat),'PRODUCT_CODE'][0],tacaia2.loc[(acc,mat),'MTRL_NO'][0],tacaia2.loc[(acc,mat),'MAT_WT'][0],acc).values())
                tacaia2.loc[(acc,mat),['MAT_VALUE','MAT_PRICE','LOST_VALUE','LOST_PRICE']]=INPUT_VALUE[:4]
                OTHER_LOST_VALUE=OTHER_LOST_VALUE+INPUT_VALUE[4]
            if acc in ['50','51']:
                tacaia2.loc[(acc,mat),['SALE_VALUE','SALE_PRICE']]=list(SalePrices(tacaia2.loc[(acc,mat),'PRODUCT_CODE'][0],tacaia2.loc[(acc,mat),'MAT_WT'][0]).values())
            
            WriteSFC(tacaia2.loc[(acc,mat),'PRODUCT_CODE'][0],acc,tacaia2.loc[(acc,mat),'MAT_VALUE'][0],tacaia2.loc[(acc,mat),'LOST_VALUE'][0],tacaia2.loc[(acc,mat),'SALE_VALUE'][0],tacaia2.loc[(acc,mat),'MAT_WT'][0])
        except:
            continue
tacacmm.iloc(tacacmm['CRRT_OTPT_ACT_AMT'].rank(ascending=False)[0])+=OTHER_LOST_VALUE

No.0
UNIT_CODE  SALE_ORDER_SUB_NO
LC01       MD2006281601         6036
Name: KEY_SEQ, dtype: int64
No.1
No.2
No.3
No.4
No.5
No.6
No.7
No.8
No.9
UNIT_CODE  SALE_ORDER_SUB_NO
LC02       MD2006281601         6038
Name: KEY_SEQ, dtype: int64
No.10
No.11
No.12
UNIT_CODE  SALE_ORDER_SUB_NO
LG01       MD2006281601         9001
Name: KEY_SEQ, dtype: int64
No.13
No.14
No.15
No.16
UNIT_CODE  SALE_ORDER_SUB_NO
LG01       MD2006281601         17075
Name: KEY_SEQ, dtype: int64
No.17
No.18
UNIT_CODE  SALE_ORDER_SUB_NO
LG01       MD2006281601         9001
Name: KEY_SEQ, dtype: int64
No.19
No.20
No.21
No.22
UNIT_CODE  SALE_ORDER_SUB_NO
LG01       MD2006281601         17075
Name: KEY_SEQ, dtype: int64
No.23
No.24
No.25
No.26
No.27
UNIT_CODE  SALE_ORDER_SUB_NO
LC02       MD2006281601         6040
Name: KEY_SEQ, dtype: int64
No.28
No.29
No.30
UNIT_CODE  SALE_ORDER_SUB_NO
LG01       MD2006281601         9003
Name: KEY_SEQ, dtype: int64
No.31
No.32
No.33
No.34
UNIT_CODE  SALE_ORDER_SUB_NO
LG01       MD2006

In [38]:
tacaia2.to_excel('tacaia2-202007-r3.xlsx')

In [40]:
print(tacaia2.loc[('01','B1804068AJAAA'),'KEY_SEQ'])

UNIT_CODE  SALE_ORDER_SUB_NO
LA03       MD2007173301         4497
Name: KEY_SEQ, dtype: int64


In [42]:
list(ProducePrices(tacaia2.loc[('01','B1804068AJAAA'),'IN_MAT_NO'][0],'01',tacaia2.loc[('01','B1804068AJAAA'),'MAT_TYPE'][0],tacaia2.loc[('01','B1804068AJAAA'),'MAT_WT'][0],tacaia2.loc[('01','B1804068AJAAA'),'MTRL_NO'][0]).values())

[1572.5549117045666,
 14.716029493772849,
 108.69801484974288,
 1.0172002138287748]

In [60]:
list(InputPrices('B2007077AC','10068Z10993','6',284.52,'31').values())[:5]

[nan, nan, 0.0, 0.0, 0.0]

In [46]:
tacaia2.loc[('31','B1804068AJAA'),'PRODUCT_CODE'][0]

'10033Z3124'

In [51]:
tacaia2.head(60)

,,,,REC_CREATOR,REC_CREATE_TIME,REC_REVISOR,REC_REVISE_TIME,ARCHIVE_FLAG,COMPANY_CODE,COMPANY_CNAME,WHOLE_BACKLOG,WHOLE_BACKLOG_ACT,JOB_CODE,...,SURFACE_PLATE_CODE,STD_CODE,UNDERLAY_CODE,SLEEVE_STYLE,MAT_VALUE,MAT_PRICE,LOST_VALUE,LOST_PRICE,SALE_VALUE,SALE_PRICE
ACCOUNT_TITLE_ITEM,MAT_NO,UNIT_CODE,SALE_ORDER_SUB_NO,,,,,,,,,,,,,,,,,,,,,
33,B1804068AJAA,LA01,MD2007173301,001161,20200803132002,,,,BH,,L0L4L1L2L3LK9A,LWL1,LA10,...,1,,,,0.000000,0.000000,0.000000,0.000000,0.0,0.0
03,B1804068AJAA,LA01,MD2007173301,001161,20200803132002,,,,BH,,L0L4L1L2L3LK9A,LWL1,LA10,...,1,,,,0.000000,0.000000,0.000000,0.000000,0.0,0.0
31,B1804068AJAA,LA01,MD2007173301,001161,20200803132002,,,,BH,,L0L4L1L2L3LK9A,LWL1,LA18,...,1,F1,AM001,,1809.482987,14.716029,108.698015,1.017200,0.0,0.0
01,B1804068AJAAA,LA03,MD2007173301,001161,20200803132002,,,,BH,,L0L4L1L2L3LK9A,LWL1L3,LA18,...,1,F1,AM001,,1572.554912,14.716029,108.698015,1.017200,0.0,0.0
50,B1804068AJAAA,LA03,MD2007173301,001161,20200803132002,,,,BH,,L0L4L1L2L3LK9A,LWL1L3,LA18,...,1,,,,0.000000,0.000000,0.000000,0.000000,0.0,0.0
01,B1804068AJAAB,LA03,MD2007173301,001161,20200803132002,,,,BH,,L0L4L1L2L3LK9A,LWL1L3,LA18,...,1,F1,AM001,,128.230060,7.964600,0.000000,0.000000,0.0,0.0
33,B1804141CB,LC02,MD2007290816,001161,20200803132002,,,,BH,,L0L4L1L2L3LCLK9A,L1LC,LC07,...,,,,,0.000000,0.000000,0.000000,0.000000,0.0,0.0
03,B1804141CB,LC02,MD2007290816,001161,20200803132002,,,,BH,,L0L4L1L2L3LCLK9A,L1LC,LC07,...,,,,,0.000000,0.000000,0.000000,0.000000,0.0,0.0
33,B1806009IDCBAA,LA01,MD2007174405,001161,20200803132002,,,,BH,,L0L4L1L2L3LJLFLK9A,L1,LA10,...,,,,,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [49]:
tacaia2.loc[('31','B1804068AJAA'),['MAT_VALUE','MAT_PRICE','LOST_VALUE','LOST_PRICE']]=list(InputPrices('B1804068AJAA','10033Z3124','3',122.96,'31').values())[:4]

In [63]:
WriteSFC('10033Z3124','31',1809.48298655431,122.96)

In [64]:
tacacm4.loc['10033Z3124']

YEAR                      2020
MON                         07
PERIOD_START_N          322.78
PRST_VRNC_AMOUNT       4750.04
CURRENT_N                    0
CRRT_OTPT_VRNC_AMT           0
OTHERS_RECEIVE_N             0
OTHR_INCM_VRNC_AMT           0
SALE_N                       0
SALE_VRNC_AMOUNT             0
NEXT_PRSP_USED_N        122.96
NEXT_PRSP_VRNC_AMT     1809.48
NEXT_MATCH_USED_N            0
NEXT_MATCH_VRNC_AMT          0
OTHERS_SEND_N                0
OTHER_SEND_VRNC_AMT          0
PERIOD_END_N                 0
PERIOD_END_VRNC_AMT          0
IN_MATCH_N                   0
IN_MATCH_VRNC_AMT            0
Name: 10033Z3124, dtype: object